# 船舶の軌道(KTモデル)にカルマンフィルタを適用する

In [ ]:
#モジュールのインポート
import numpy as np
import matplotlib.pyplot as plt

: 

## KTモデルシミュレーションに基づいた関数を生成する

$T\dot{r}+r=K\delta$

$T$:追従性指数[sec], $r$:旋回角速度[rad/sec], $K$:旋回性指数[1/sec], $\delta$:舵角[rad]

舵角20度一定の旋回運動について考える

In [ ]:
#システムモデルの関数を定義する
def f(dt,parameter):
    parameter[0]=(parameter[1]*parameter[3]*dt+parameter[2]*parameter[0])/(parameter[2]+dt)
    return parameter
    
#逆関数
def f_inv(dt,parameter):
    parameter[0]=((parameter[2]+dt)*parameter[0]-parameter[1]*parameter[3]*dt)/parameter[2]
    return parameter


#KTモデルの真値とシステム値を算出する関数
def KT_generetor(v,w,dt_true,dt,time,parameter):

    R_true=[[],[]] #時間と回頭角速度の真値リスト
    R_system=[[],[]] #時間と回頭角速度の状態値リスト
    R_observation=[[],[]]
    
    for t in range(int(time/dt_true)):

        parameter=f(dt_true,parameter) 

        r=parameter[0]

        r_sysytem=np.random.normal(loc=r,scale=v) #システムモデル
        
        R_true[0].append(t*dt_true)
        R_true[1].append(r)

        R_system[0].append(t*dt_true)
        R_system[1].append(r_sysytem)
        
        if t%(dt/dt_true)==0:

            r_observation=np.random.normal(loc=r_sysytem,scale=w)
            R_observation[0].append(t*dt_true)
            R_observation[1].append(r_observation)
            R_observation[1][0]=0


    return R_true,R_system,R_observation


#回頭角速度rを描画する関数
def draw_r(R_true,R_observation,R_kalman,v,w,dt):
    fig = plt.figure(dpi=130)


    plt.scatter(R_observation[0],R_observation[1],label="observed",s=3,c="r")
    if type(R_kalman) is list:
        plt.plot(R_kalman[0],R_kalman[1],label="kalman",c="m",ls="-")
    plt.plot(R_true[0],R_true[1],label="true",c="k")

    plt.title(f"angular rate of speed (v={v},w={w},dt={dt})")
    plt.ylim(-0.20,0.25)
    plt.xlabel('t [s]')
    plt.ylabel('r [rad/s]')
    plt.legend(bbox_to_anchor=(0.25, 1),loc='upper right', borderaxespad=1, fontsize=8)
    plt.show()


#カルマンフィルター
def kalman_filter(R_observation,parameter,V,v,w,dt):

    V_list=[[],[]]
    R_kalman=[[],[]]
    G_list=[[],[]]

    for t,r_ob in zip(R_observation[0],R_observation[1]):

        #prediction
        R_kalman[0].append(t)
        R_kalman[1].append(parameter[0])
        
        V_list[0].append(t)
        V_list[1].append(V)

        _r_=f(dt,parameter) #次時点の予測値
        V=V+v

        G=V/(V+w)

        G_list[0].append(t)
        G_list[1].append(G)

        parameter[0]=_r_[0]+G*(r_ob-_r_[0])
        V=V-G*V

    R_kalman[1][0]=0

    return R_kalman,V_list,G_list


#dtを変化させたときの真値と状態推定値の差分を計算し描画する関数
def diff_kalman_true_dt(v,w,V,dt_true,dt_list,time,parameter,random_times):
    diff_list=[]

    for dt in dt_list:

        diff_random=0
        
        for i in range(random_times):
            np.random.seed(i)
            parameter[0]=0
            R_true,R_system,R_observation=KT_generetor(v,w,dt_true,dt,time,parameter)
            R_kalman,V_list,G_list=kalman_filter(R_observation,parameter,V,v,w,dt)
            
            #状態推定のグラフを1枚だけ出力
            if i==1:
                draw_r(R_true,R_observation,R_kalman,v,w,dt)

            diff=0

            for t in range(len(R_kalman[0])):
                diff+=abs(R_kalman[1][t]-R_true[1][t*int(dt/dt_true)])

            diff_random+=diff*dt

        diff_list.append(diff_random/random_times)
        

    fig = plt.figure(dpi=130)
    plt.tick_params(labelsize=6)
    plt.bar([str(n) for n in dt_list],diff_list)
    plt.title(f'Difference between true r and r with Kalman filter applied\n (v={v},w={w},random_times={random_times})')
    plt.ylabel('difference[rad/s]')
    plt.xlabel('dt[s]')
    plt.show()
    print(diff_list)


#wを変化させたときの真値と状態推定値の差分を計算して描画する関数
def diff_kalman_true_w(v,w_list,V,dt_true,dt,time,parameter,random_times):
    
    diff_list=[]
    
    R_true_list=[]
    R_system_list=[]
    R_observation_list=[]
    R_kalman_list=[]

    for w in w_list:
        
        diff_random=0
        for i in range(random_times):
            np.random.seed(i)
            parameter[0]=0
            R_true,R_system,R_observation=KT_generetor(v,w,dt_true,dt,time,parameter)
            R_kalman,V_list,G_list=kalman_filter(R_observation,parameter,V,v,w,dt)

            #状態推定のグラフを1枚だけ出力
            if i==0:
                draw_r(R_true,R_observation,R_kalman,v,w,dt)

            diff=0

            for t in range(len(R_kalman[0])):
                diff+=abs(R_kalman[1][t]-R_true[1][t*int(dt/dt_true)])

            diff_random+=diff*dt

        diff_list.append(diff_random/random_times)
        

    fig = plt.figure(dpi=130)
    plt.tick_params(labelsize=6)
    plt.bar([str(n) for n in w_list],diff_list)
    plt.title(f'Difference between true r and r with Kalman filter applied\n(v={v},dt={dt},random_times={random_times})')
    plt.ylabel('difference [rad/s]')
    plt.xlabel('Observation noise w[rad/s]')
    plt.show()
    print(diff_list)


#変更する必要あり！
#rをx,yに変換する関数
def chenge_r2xy(r_list,dt):

    psi0=0
    
    #観測値のリストを作成
    psi_list=[]
    dx_list=[]
    dy_list=[]
    x_list=[]
    y_list=[]

    for i in r_list[1]:
        psi_list.append(psi0)
        psi0+=i*dt
        

    for i in psi_list:
        dx_list.append(u0*np.cos(i))

    for i in psi_list:
        dy_list.append(u0*np.sin(i))
    
    x0=0
    y0=0
    
    for i in dx_list:
        x_list.append(x0)
        x0+=i*dt
        
    
    for i in dy_list:
        y_list.append(y0)
        y0+=i*dt
        


    return psi_list,x_list,y_list

    

: 

In [ ]:
#初期値
r=0.0 * np.pi / 180 #回頭角速度[rad/s]
K_true = 0.15 #KTモデルのパラメータ
T_true = 60 #KTモデルのパラメータ
delta=20*np.pi/180 #舵角[rad]
parameter=[r,K_true,T_true,delta]
dt_true=0.01 #真値の観測幅[s]
dt=0.5
dt_list=[10,5,2,1,0.5,0.75,0.1,0.05]#観測値の観測幅[s]
time=150 #観測時間[s]
v=0.01
w=0.1
w_list=[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08]
V=1
random_times=50


: 

In [ ]:
diff_kalman_true_dt(v,w,V,dt_true,dt_list,time,parameter,random_times)
diff_kalman_true_w(v,w_list,V,dt_true,dt,time,parameter,random_times)

: 

In [ ]:
#初期値
r=0.0 * np.pi / 180 #回頭角速度[rad/s]
K_true = 0.15 #KTモデルのパラメータ
T_true = 60 #KTモデルのパラメータ
delta=20*np.pi/180 #舵角[rad]
parameter=[r,K_true,T_true,delta]
dt_true=0.01 #真値の観測幅[s]
dt=0.1
dt_list=[10,5,2.5,1,0.5,0.1]#観測値の観測幅[s]
time=100 #観測時間[s]
v=0.01
#w=0.1
w_list=[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08]
V=1
random_times=10
psi0=0
x0=0
y0=0
u0= 15 * (1852.0 / 3600)

: 

In [ ]:
#回頭角速度から回頭角、航路を算出する
for dt in dt_list:
    for i in range(random_times):
        np.random.seed(i)
        
        #初期値
        r=0.0 * np.pi / 180 #回頭角速度[rad/s]
        K_true = 0.15 #KTモデルのパラメータ
        T_true = 60 #KTモデルのパラメータ
        delta=20*np.pi/180 #舵角[rad]
        parameter=[r,K_true,T_true,delta]
        dt_true=0.01 #真値の観測幅[s]
        w=0.05
        time=150 #観測時間[s]
        v=0.01
        V=1
        random_times=10
        psi0=0
        x0=0
        y0=0
        u0= 15 * (1852.0 / 3600)

        R_true,R_system,R_observation=KT_generetor(v,w,dt_true,dt,time,parameter)
        R_kalman,V_list,G_list=kalman_filter(R_observation,parameter,V,v,w,dt)
        psi_true,x_true,y_true=chenge_r2xy(R_true,dt_true)
        psi_observation,x_observation,y_observation=chenge_r2xy(R_observation,dt)
        psi_kalman,x_kalman,y_kalman=chenge_r2xy(R_kalman,dt)

        draw_r(R_true,R_observation,R_kalman,v,w,dt)
        
        fig = plt.figure(dpi=130)
        plt.plot(R_true[0],psi_true,c="k",label="true")
        plt.plot(R_observation[0],psi_observation,label="observation")
        plt.plot(R_kalman[0],psi_kalman,c="r",label="kalman")
        plt.legend()
        plt.grid(True)
        plt.show()

        fig = plt.figure(dpi=130)
        #plt.plot(x_true,y_true,c="k",label="true")
        #plt.plot(x_observation,y_observation,c="r",label="true")
        plt.plot(x_kalman,y_kalman,c="m",label="kalman")
        plt.plot(x_observation,y_observation,c="r",label="observation")
        plt.plot(x_true,y_true,c="k",label="true")
        plt.title(f'Ship tracks dt={dt}[s],v={v}[rad/s],w={w}[rad/s]')
        plt.xlabel('x[m]')
        plt.ylabel('y[m]')
        plt.legend()
        plt.grid(True)
        plt.show()

: 